In [11]:
!ls /home/sagemaker-user/.cache/pip

http-v2  selfcheck  wheels


In [12]:
!pip cache purge

Files removed: 0


In [13]:
!pip install sc-toolbox pertpy

In [72]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import random
import sc_toolbox
import pertpy 

import itertools 

import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [18]:
%%R
BiocManager::install('MAST')

* installing *source* package ‘MAST’ ...
** using staged installation
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (MAST)


Update all/some/none? [a/s/n]: 

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cloud.r-project.org
Bioconductor version 3.14 (BiocManager 1.30.25), R 4.1.2 (2021-11-01)
Installing package(s) 'MAST'
trying URL 'https://bioconductor.org/packages/3.14/bioc/src/contrib/MAST_1.20.0.tar.gz'
Content type 'application/octet-stream' length 6728468 bytes (6.4 MB)
downloaded 6.4 MB


The downloaded source packages are in
	‘/tmp/RtmpWZpo9N/downloaded_packages’
Installation paths not writeable, unable to update packages
  path: /usr/lib/R/library
  packages:
    boot, class, cluster, codetools, foreign, KernSmooth, lattice, mgcv, nlme,
    nnet, rpart, spatial, survival
  path: /usr/local/lib/R/site-library
  packages:
    caTools, ggrepel, reticulate, spatstat.univar
Old packages: 'matrixStats', 'Seurat', 'SeuratObject'


In [19]:
%%R
library(edgeR)
library(MAST)

Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOr

In [29]:
adata = pertpy.data.srivatsan_2020_sciplex2()

Output()

In [31]:
adata.obs

,ncounts,hash_umis,pval_demultiplexing,qval_demultiplexing,top_to_second_best_ratio,top_oligo,perturbation,dose_value,well,celltype,...,cancer,disease,tissue_type,organism,perturbation_type,ngenes,percent_mito,percent_ribo,nperts,chembl-ID
cell_barcode,,,,,,,,,,,,,,,,,,,,,
A01_A01_RT_384,1939,NaN,NaN,NaN,NaN,NaN,control,NaN,NaN,alveolar basal epithelial cells,...,True,lung adenocarcinoma,cell_line,human,drug,1258,11.913358,1.856627,1,NaN
A01_A01_RT_389,1234,8.0,1.000000e+00,1.000000e+00,7.000000,Dex_0.1_BC07,Dex,0.1,BC07,alveolar basal epithelial cells,...,True,lung adenocarcinoma,cell_line,human,drug,813,17.828201,1.620746,1,NaN
A01_A01_RT_390,3430,16.0,1.404159e-140,3.650813e-139,15.005464,Nutlin_0.5_AB02,Nutlin,0.5,AB02,alveolar basal epithelial cells,...,True,lung adenocarcinoma,cell_line,human,drug,2014,8.921283,1.078717,1,CHEMBL407632;CHEMBL191334
A01_A01_RT_396,6512,35.0,0.000000e+00,0.000000e+00,2.379168,Dex_50_BC10,Dex,50,BC10,alveolar basal epithelial cells,...,True,lung adenocarcinoma,cell_line,human,drug,3222,4.545455,1.320639,1,NaN
A01_A01_RT_397,5354,15.0,5.487454e-276,2.030358e-274,5.789614,BMS_0.5_AH10,BMS,0.5,AH10,alveolar basal epithelial cells,...,True,lung adenocarcinoma,cell_line,human,drug,2804,11.953679,1.849085,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H12_B02_RT_752,12956,63.0,0.000000e+00,0.000000e+00,4.731265,BMS_0.1_BG07,BMS,0.1,BG07,alveolar basal epithelial cells,...,True,lung adenocarcinoma,cell_line,human,drug,4762,8.891633,1.011115,1,NaN
H12_B02_RT_753,6199,85.0,0.000000e+00,0.000000e+00,81.158163,Nutlin_0.1_AA07,Nutlin,0.1,AA07,alveolar basal epithelial cells,...,True,lung adenocarcinoma,cell_line,human,drug,3078,6.065495,1.839006,1,CHEMBL407632;CHEMBL191334
H12_B02_RT_760,5190,50.0,0.000000e+00,0.000000e+00,38.865385,Dex_5_AC12,Dex,5,AC12,alveolar basal epithelial cells,...,True,lung adenocarcinoma,cell_line,human,drug,2591,12.003854,2.312139,1,NaN


In [42]:
pd.DataFrame(adata.X.todense())

,0,1,2,3,4,5,6,7,8,9,...,35906,35907,35908,35909,35910,35911,35912,35913,35914,35915
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24257,0.0,0.0,2.0,2.0,0.0,0.0,4.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24258,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24260,1.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,7.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
adata.layers["counts"]   = adata.X.copy()
sc.pp.filter_cells(adata, min_genes = 200)
sc.pp.filter_genes(adata, min_cells = 3)

**Design Considerations**
1. 1 Cell Line (A549)
2. 4 Compounds
3. 7 Different Doses In Triplicates
4. 84 Drug-Dose-Replciate Combinations

In [88]:
doses          = adata.obs['dose_value'].unique()
perturbations  = adata.obs['perturbation'].unique()  # Drug Pertubation or Control


combinations   = itertools.product(doses, perturbations)

adata.obs['Condition'] = [str(adata.obs['dose_value'][index]) + ' ' +  str(adata.obs['perturbation'][index]) 
                        if str(adata.obs['dose_value'][index]) != 'nan' else str(adata.obs['perturbation'][index])
                        for index in adata.obs.index]
                        

# for combination in combinations:
#     dose         = combination[0]
#     perturbation = combination[1]
    # adata[(adata.obs['dose_value'] == dose) & (adata.obs['perturbation'] == perturbation)]

In [89]:
adata.obs['Condition']

cell_barcode
A01_A01_RT_384       control
A01_A01_RT_389       0.1 Dex
A01_A01_RT_390    0.5 Nutlin
A01_A01_RT_396        50 Dex
A01_A01_RT_397       0.5 BMS
                     ...    
H12_B02_RT_752       0.1 BMS
H12_B02_RT_753    0.1 Nutlin
H12_B02_RT_760         5 Dex
H12_B02_RT_761       10 SAHA
H12_B02_RT_763    0.1 Nutlin
Name: Condition, Length: 24262, dtype: object

In [ ]:
adata_hvg = adata.copy()
sc.pp.highly_variable_genes(adata_hvg)